In generative adversarial network (GAN), convolutions and transposed convolutions are both heavily involved. While convolutions play an important role in the discriminator, transposed convolutions are the primary building blocks for the generator. The tensorflow API - Keras - has made building GAN a very convenient process. However, sometimes it can be confusing of **what values should be used for the kernel size, strides, and padding to yield the right output shapes.** Setting the right values for the parameters require us to understand how transposed convolutions work. In this notebook, I would like to share some of my personal understandings about transposed convolutions. Throughout the notebook, I will use convolutions as the comparison to better explain transposed convolutions. I will also show you how I implement these understanding to build my own convolutional and transposed convolutional layers, which act as a naive version of the Conv2D and Conv2DTranspose layers from Keras. The notebook consists of three sections:

#### 1. What are transposed convolutions?
#### 2. How do the parameters (kernel size, strides, and padding) affect transposed convolutions?
#### 3. Build my own Conv2D and Conv2DTranspose layers from scratch

*Note: this notebook does not explain transposed convolution through mathematical derivations. Instead, I will use mainly pictures to visually explain the concepts, as I found it is easier to understand transposed convolution through some visual aids.*

# Section 1: What are transposed convolutions?

I understand transposed convolutions as the opposite of convolutions. In convolutional layer, we use a special operation named cross-correlation (in machine learning, it is more often known as convolution, and thus the layers are named "Convolutional Layers") to calculate the output values. This operation adds all the neighboring numbers in the input layer together, weighted by a [convolution matrix](https://en.wikipedia.org/wiki/Kernel_(image_processing)#:~:text=In%20image%20processing%2C%20a%20kernel,a%20kernel%20and%20an%20image.) (kernel). For example, in the image below, the output value **55** is calculated by the element-wise multiplication between the 3x3 part of the input layer and the 3x3 kernel, and sum all results together:

$$ 3\times1 + 5\times2 + 2\times1 + 4\times2 + 1\times1 + 3\times2 + 6\times1 + 3\times1 + 8\times2 = 55 $$

![image](Images/Convolutional.png)

Without any padding, this operation transforms a 4x4 matrix into a 2x2 matrix. This looks like someone is casting the light from left to right, and projecting an object (the 4x4 matrix) through a hole (the 3x3 kernel), and yield a smaller object (the 2x2 matrix). **Now, our question is: what if we want to go backwards from a 2x2 matrix to a 4x4 matrix?** Well, the intuitive way is, we just cast the light backwards! Mathematically, instead of multiplying two 3x3 matrices, we can multiply each value in the input layer by the 3x3 kernel to yield a 3x3 matrix. Then, we just combine all of them together according to the initial positions in the input layer, and sum the overlapped values together:

**Step 1: multiply each element in the input layer by the kernel**

![image](images/Convolutional_Transpose.PNG)

**Step 2: combine all results together**

![image](images/Convolutional_Transpose_2.PNG)

In this way, it is always certain that the output of the transposed convolution operation can have exactly the same shape as the input of the previous convolution operation, because we just did exactly the reverse. However, you may notice that the numbers are not restored. Therefore, a totally different kernel has to be used to restore the initial input matrix, and this kernel can be determined through training. 

To demonstrate that my results are not just some random numbers, I build the convolutional neural networks using the conditions indicated above through Keras. As can be seen from the code below, the outputs are exactly the same. 

In [1]:
from tensorflow import keras
import numpy as np

In [2]:
X = np.array([[3, 5, 2, 7], [4, 1, 3, 8], [6, 3, 8, 2], [9, 6, 1, 5]])
X = X.reshape(1, 4, 4, 1)

In [3]:
model_Conv2D = keras.models.Sequential()
model_Conv2D.add(keras.layers.Conv2D(1, (3, 3), strides=(1, 1), padding='valid', input_shape=(4, 4, 1)))
weights = [np.asarray([[[[1]], [[2]], [[1]]], [[[2]], [[1]], [[2]]], [[[1]], [[1]], [[2]]]]), np.asarray([0])]
model_Conv2D.set_weights(weights)
yhat = model_Conv2D.predict(X)
yhat.reshape(2, 2)

array([[55., 52.],
       [57., 50.]], dtype=float32)

In [4]:
X = yhat
model_Conv2D = keras.models.Sequential()
model_Conv2D.add(keras.layers.Conv2DTranspose(1, (3, 3), strides=(1, 1), padding='valid', input_shape=(2, 2, 1)))
weights = [np.asarray([[[[1]], [[2]], [[1]]], [[[2]], [[1]], [[2]]], [[[1]], [[1]], [[2]]]]), np.asarray([0])]
model_Conv2D.set_weights(weights)
yhat = model_Conv2D.predict(X)
yhat.reshape(4, 4)

array([[ 55., 162., 159.,  52.],
       [167., 323., 319., 154.],
       [169., 264., 326., 204.],
       [ 57., 107., 164., 100.]], dtype=float32)

## Why "Transposed"?

Now that you may wondering: hey, this looks just like a reversed convolution. Why is it named "transposed" convolution? 

To be honest, I don't know why I had to struggle with this question, but I did. I believed that it's named as "transposed" convolution for a reason. To answer this question, I read many online resources about transposed convolution. An article named ["Up-sampling with Transposed Convolution"](https://medium.com/activating-robotic-minds/up-sampling-with-transposed-convolution-9ae4f2df52d0) helped me a lot. In this article, the author Naoki Shibuya expresses the convolution operation using **a zero-padded convolution matrix instead of a normal squared-shape convolution matrix**. Essentially, instead of expressing the above kernel as a 3x3 matrix, when performing convolutional transformation, we can express it as a 4x16 matrix. And instead of expressing the above input as a 4x4 matrix, we can express it as a 16x1 vector:

**Expressing 3x3 kernel as 4x16 convolution matrix:**

4 rows: in total we can perform four convolutions by spliting a 4x4 input matrix into four 3x3 matrces;

16 columns: the input matrix will be transformed into a 16x1 vector. To perform the matrix multiplication, it has to be 16 columns.

![image](images/Convolution_Matrix.PNG)

**Expressing 4x4 input matrix as 16x1 input vector:**

![image](Images/Input_Vector.PNG)

In this way, we can directly perform the matrix multiplication to get an output layer. The reshaped output layer will be exactly the same as the one derived by the general convolution operation.

![image](Images/Matrix_Multiplication.PNG)

**Now it comes the most interesting part!** When we perform transposed convolution operation, we just simply transpose the zero-padded convolution matrix and multiply it with the input vector (which is the output of the convolutional layer). In the picture below, the four colored vectors in the middle stage represent the intermediate step of the matrix multiplication:

![image](Images/Transpose_Matrix_Multiplication.PNG)

If we rearrange the four vectors in the middle stage, we will get the four 4x4 matrices that have exactly the same numbers as the 3x3 matrices we obtained by multiplying the 3x3 kernel with each individual element in the input layer,  with the extra slots filled by zeros. These four matrices can also be further combined to get the final 4x4 output matrix:

**Rearrange the four 16x1 vectors into 4x4 matrices:**

![image](Images/Four_Matrices.PNG)

**These four matrices can be further combined to give us the final output layer:**

![image](Images/Four_Matrices_Combine.PNG)

**Thus, the operation is called "transposed" convolution because we performed exactly the same operation except that we transposed the convolution matrix!**

# Section 2: How do the parameters (kernel size, strides, and padding) affect transposed convolutions?

## Kernel size

**In convolutions**, the kernel size affects how many numbers in the input layer you "project" to form one number in the output layer. The larger the kernel size, the more numbers you use, and thus each number in the output layer is a broader representation of the input layer and carries more information from the input layer. But at the same time, using a larger kernel will give you an output with smaller size. For example, a 4x4 input matrix with a 3x3 kernel will yield a 2x2 output matrix, while with a 2x2 kernel will yield a 3x3 output matrix (if no padding is added):

![image](Images/Convolution_Kernel_Size.PNG)

**In transposed convolutions**, as it works the opposite as convolutions, the larger the kernel size, the larger the output matrix (without padding), as we "disperse" each single number from the input layer to a broader area to form a larger matrix: 

![image](Images/Transpose_Convolution_Kernel_Size.PNG)

## Strides

**In convolutions**, strides parameter indicate how fast the kernel moves along the rows and columns on the **input layer**.  If a stride is (1, 1), the kernel moves one row/column for each step; if a stride is (2, 2), the kernel moves two rows/columns for each step. As a result, the larger the strides, the smaller the output matrix (without padding).

![image](Images/Convolution_Strides.PNG)

**In transposed convolutions**, strides parameter indicates how fast the kernel moves on the **output layer**, as explained by the picture below. Notice that the kernel always move only one number at a time on the input layer. Thus, the larger the strides, the larger the output matrix (without padding).

![image](Images/Transpose_Convolution_Strides.PNG)

## Padding

**In convolutions**, we often want to maintain the shape of the input layers, and we do it through zero-padding. In Keras, padding parameter can be one of two strings: "valid" or "same". When padding is "valid", it means no zero-padding is implemented. When padding is "same", the input-layer is padded in a way so that the output layer has a same shape as the input layer.

![image](Images/Convolution_Padding.PNG)

**In transposed convolutions**, the padding parameter also can be the two strings: "valid" and "same". However, since we expand the input layer in transposed convolutions, if choosing "valid", the output shape will be larger than the input shape. If "same" is used, then only the very middle part of the output that has the same shape as the input is maintained. An easier way to remember "valid" and "same" in both convolutions and transposed convolutions is:
- "valid": no extra operation is performed. The output stays what it is meant to be.
- "same": the output always has the same shape as the input. In convolutions, because the output is meant to be smaller, extra zeros are added; in transposed convolutions, because the output is meant to be larger, the output is cut into a smaller shape.

![image](Images/Transpose_Convolution_Padding.PNG)

# Build My Own Conv2D and Conv2DTransposed Layers From Scratch

Up to now, I hope you have a better understanding of what transposed convolutions are and what do parameters do. They may still be very abstract for you. But don't worry, now we can get our hands dirty and build our own convolutional and transposed convolutional layers using these principles - this will definitely unravel the mistery of convolutions and transposed convolutions!

**Let's start with Conv2D layer:**

In [5]:
from math import floor, ceil
def Conv2D(X, W, padding="valid", strides=(1, 1)):
    # Define length of zero-padding
    if padding == "same":
        # returns the output with the same shape as the input
        p_row = ceil(((X.shape[0] - 1) * strides[0] + W.shape[0] - X.shape[0])/2)
        p_col = ceil(((X.shape[1] - 1) * strides[1] + W.shape[1] - X.shape[1])/2)
    elif padding == "valid":
        # returns the output without any padding
        p_row = 0
        p_col = 0
    # Define input after paddings
    row_num = X.shape[0] + 2 * p_row
    col_num = X.shape[1] + 2 * p_col
    X_padded = np.zeros(shape=(row_num, col_num))
    X_padded[p_row:p_row+X.shape[0], p_col:p_col+X.shape[1]] = X
    # Calculate the output
    output = []
    for i in range(0, floor((X_padded.shape[0] - W.shape[0])/strides[0])+1, strides[0]):
        output.append([])
        for j in range(0, floor((X_padded.shape[1] - W.shape[1])/strides[1])+1, strides[1]):
            X_sub = X_padded[i:i+W.shape[0], j:j+W.shape[1]] # Subset of X
            output[-1].append(np.sum(X_sub * W))
    return(np.array(output))

Let's go through my home-made Conv2D layer:
- First, I defined the number of zero-paddings I need to add. If the padding is "valid", then I don't need to add any padding. If the padding is "same", I calculate the number of padding on each side of the input layer based on the formula:

$$ p = ceil\left(\frac{(o - 1)s + m - n}{2}\right) $$

Where:
- $o$ is the output size
- $s$ is the strides
- $m$ is the kernel size
- $n$ is the input size
- $p$ is the padding number on each side

This formula is derived from the formula for calculating the output shape:

$$ o = floor\left(\frac{n + 2p -m}{s}\right)+1 $$

As the output size has to be the same as the input size, $o = n$.

- Then, I padded the input by constructing a larger matrix filled with zeros and put the original input in the middle.
- After that, I calculated the output using convolution operation. The convolution operation is performed between the kernel W and the subset of input X_sub (which has the same shape as the kernel). The output indices *i, j* range from 0 to $o$, where $o$ is the output shape defined above. A graphical explaination of the process can be found below:

![image](Images/Conv2D_Code_Explain.PNG)

**My Conv2D works the same as the Cov2D layer in Keras:**

In [6]:
X = np.array([[3, 5, 2, 7], [4, 1, 3, 8], [6, 3, 8, 2], [9, 6, 1, 5]])
X_reshape = X.reshape(1, 4, 4, 1)
W = np.array([[1, 2, 1], [2, 1, 2], [1, 1, 2]])

In [7]:
Conv2D(X, W, padding="valid", strides=(1, 1))

array([[55., 52.],
       [57., 50.]])

In [8]:
model_Conv2D = keras.models.Sequential()
model_Conv2D.add(keras.layers.Conv2D(1, (3, 3), strides=(1, 1), padding='valid', input_shape=(4, 4, 1)))
weights = [np.asarray([[[[1]], [[2]], [[1]]], [[[2]], [[1]], [[2]]], [[[1]], [[1]], [[2]]]]), np.asarray([0])]
model_Conv2D.set_weights(weights)
yhat = model_Conv2D.predict(X_reshape)
yhat.reshape(2, 2)

array([[55., 52.],
       [57., 50.]], dtype=float32)

In [9]:
Conv2D(X, W, padding="same", strides=(1, 1))

array([[19., 26., 46., 22.],
       [29., 55., 52., 40.],
       [42., 57., 50., 43.],
       [36., 46., 44., 19.]])

In [10]:
model_Conv2D = keras.models.Sequential()
model_Conv2D.add(keras.layers.Conv2D(1, (3, 3), strides=(1, 1), padding='same', input_shape=(4, 4, 1)))
weights = [np.asarray([[[[1]], [[2]], [[1]]], [[[2]], [[1]], [[2]]], [[[1]], [[1]], [[2]]]]), np.asarray([0])]
model_Conv2D.set_weights(weights)
yhat = model_Conv2D.predict(X_reshape)
yhat.reshape(4, 4)

array([[19., 26., 46., 22.],
       [29., 55., 52., 40.],
       [42., 57., 50., 43.],
       [36., 46., 44., 19.]], dtype=float32)

**Now let's build the transposed convolutional layer:**

In [11]:
from math import floor, ceil
def Conv2DTranspose(X, W, padding="valid", strides=(1, 1)):
    # Define length of padding
    if padding == "same":
        # returns the output with the same shape as the input
        p_row = ((X.shape[0] - 1) * strides[0] + W.shape[0] - X.shape[0])/2
        p_col = ((X.shape[1] - 1) * strides[1] + W.shape[1] - X.shape[1])/2
    elif padding == "valid":
        # returns the output without any padding
        p_row = 0
        p_col = 0
    # Define output shape
    row_num = (X.shape[0] - 1) * strides[0] + W.shape[0]
    col_num = (X.shape[1] - 1) * strides[1] + W.shape[1]
    output = np.zeros([row_num, col_num])
    # Calculate the output
    for i in range(0, X.shape[0]):
        i_prime = i * strides[0] # Index in output
        for j in range(0, X.shape[1]):
            j_prime = j * strides[1]
            # Insert values
            for k_row in range(W.shape[0]):
                for k_col in range(W.shape[1]):
                    output[i_prime+k_row, j_prime+k_col] += W[k_row, k_col] * X[i, j]
    # Add padding
    output_padded = output[floor(p_row):output.shape[0]-ceil(p_row), floor(p_col):output.shape[0]-ceil(p_col)]
    return(np.array(output_padded))

Let's against break up the code:
- The number of padding is defined using the same method, but without applying the *ceil* function 
- The output shape is defined by the formula below:

$$ o = (n - 1)s+m $$

If you compare to the fomula to calculate the output shape of Conv2D, you can notice that in Conv2DTranspose both the strides and the kernel size have opposite effect on the output shape.

- To calculate the output, I use two pairs of indices: *i, j* move along the input and *i_prime, j_prime* move along the output. When *i, j* change, *i_prime, j_prime* change with step sizes of given strides. For example, if *i* changes from 0 to 1, and strides = (2, 2), then *i_prime* changes from 0 to 2. Each value in the input matrix is multiplied by every value in the kernel, and the results are recorded in the output matrix. 
- Lastly, the output is padded by only selecting the matrix in the middle that has the same shape as the input matrix. Note that I used both *floor* and *ceil* functions on *p_row* and *p_col*. This ensures the output shape remains correct when *p_row* and *p_col* are not integers.

A graphically explaination for the process of calculating the output is shown below:

![image](Images/Conv2DTranspose_Code_Explain_1.PNG)

![image](Images/Conv2DTranspose_Code_Explain_2.PNG)

![image](Images/Conv2DTranspose_Code_Explain_3.PNG)

![image](Images/Conv2DTranspose_Code_Explain_4.PNG)

**Now we can verify our function by comparing the results with Conv2DTranspose in Keras:**

In [12]:
X = np.array([[55, 52], [57,50]])
X_reshape = X.reshape(1, 2, 2, 1)
W = np.array([[1, 2], [2, 1]])

In [13]:
Conv2DTranspose(X, W, padding="valid", strides=(1, 1))

array([[ 55., 162., 104.],
       [167., 323., 152.],
       [114., 157.,  50.]])

In [14]:
model_Conv2D_Transpose = keras.models.Sequential()
model_Conv2D_Transpose.add(keras.layers.Conv2DTranspose(1, (2, 2), strides=(1, 1), padding='valid', input_shape=(2, 2, 1)))
weights = [np.asarray([[[[1]], [[2]]], [[[2]], [[1]]]]), np.asarray([0])]
model_Conv2D_Transpose.set_weights(weights)
yhat = model_Conv2D_Transpose.predict(X_reshape)
yhat.reshape(3, 3)

array([[ 55., 162., 104.],
       [167., 323., 152.],
       [114., 157.,  50.]], dtype=float32)

In [15]:
Conv2DTranspose(X, W, padding="same", strides=(1, 1))

array([[ 55., 162.],
       [167., 323.]])

In [16]:
model_Conv2D_Transpose = keras.models.Sequential()
model_Conv2D_Transpose.add(keras.layers.Conv2DTranspose(1, (2, 2), strides=(1, 1), padding='same', input_shape=(2, 2, 1)))
weights = [np.asarray([[[[1]], [[2]]], [[[2]], [[1]]]]), np.asarray([0])]
model_Conv2D_Transpose.set_weights(weights)
yhat = model_Conv2D_Transpose.predict(X_reshape)
yhat.reshape(2, 2)

array([[ 55., 162.],
       [167., 323.]], dtype=float32)

In [17]:
Conv2DTranspose(X, W, padding="valid", strides=(2, 2))

array([[ 55., 110.,  52., 104.],
       [110.,  55., 104.,  52.],
       [ 57., 114.,  50., 100.],
       [114.,  57., 100.,  50.]])

In [18]:
model_Conv2D_Transpose = keras.models.Sequential()
model_Conv2D_Transpose.add(keras.layers.Conv2DTranspose(1, (2, 2), strides=(2, 2), padding='valid', input_shape=(2, 2, 1)))
weights = [np.asarray([[[[1]], [[2]]], [[[2]], [[1]]]]), np.asarray([0])]
model_Conv2D_Transpose.set_weights(weights)
yhat = model_Conv2D_Transpose.predict(X_reshape)
yhat.reshape(4, 4)

array([[ 55., 110.,  52., 104.],
       [110.,  55., 104.,  52.],
       [ 57., 114.,  50., 100.],
       [114.,  57., 100.,  50.]], dtype=float32)

As expected, we got our results aligned with the results produced by Keras!